In [6]:
using DataFrames
using CSV
using Plots
using Distributions
using JLD2

In [7]:
include("CommonFunctions.jl")
include("CommonConstants.jl")
using .CommonFunctions
using .CommonConstants

Load up the betas data and have a look at the formatting.

In [8]:
options_folder = option_foldername(currency_type="local", strict_eq=true)

"local-rets_eq-strict"

In [9]:
betas_filename = joinpath("../data/results", options_folder, "betas.jld2")
betas, data_description = load(betas_filename, "data", "data_description");

(3612206×17 DataFrame
     Row │ fundid      date        world_capm_lrv_net_betas           world_capm_lrv_betas               world_ff3_verdelhan_betas          world_ffcarhart_lrv_betas          world_ff3_lrv_betas                world_ffcarhart_verdelhan_betas    world_ff3_lrv_net_betas            world_ffcarhart_lrv_net_betas      world_ff5_verdelhan_betas          world_ff6_lrv_betas                world_ff6_verdelhan_betas          world_ff6_lrv_net_betas            world_capm_verdelhan_betas         world_ff5_lrv_net_betas            world_ff5_lrv_betas               
         │ String15    Date        Tuple…                             Tuple…                             Tuple…                             Tuple…                             Tuple…                             Tuple…                             Tuple…                             Tuple…                             Tuple…                             Tuple…                             Tuple…                           

In [10]:
display(first(betas, 5))

Row,fundid,date,world_capm_lrv_net_betas,world_capm_lrv_betas,world_ff3_verdelhan_betas,world_ffcarhart_lrv_betas,world_ff3_lrv_betas,world_ffcarhart_verdelhan_betas,world_ff3_lrv_net_betas,world_ffcarhart_lrv_net_betas,world_ff5_verdelhan_betas,world_ff6_lrv_betas,world_ff6_verdelhan_betas,world_ff6_lrv_net_betas,world_capm_verdelhan_betas,world_ff5_lrv_net_betas,world_ff5_lrv_betas
,String15,Date,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,FS00008KO2,2013-09-30,"([55.4548, 0.10777, -0.29293], [12.9151, 0.13787, 0.40625], 21.0)","([54.1408, -0.05196, 0.07412], [13.9236, 0.15435, 0.35567], 21.0)","([39.97, 48.9912, 9.54404, 0.02954, -0.26623], [21.4814, 0.1507, 0.36301], 19.0)","([38.9222, 53.1433, 23.31, 8.74302, -0.05529, 0.2978], [15.1689, 0.15918, 0.37209], 18.0)","([39.7977, 49.1354, 9.58741, -0.03105, 0.27168], [21.4732, 0.15081, 0.36277], 19.0)","([39.1395, 52.9508, 23.1258, 8.66574, 0.05319, -0.29092], [15.1656, 0.15892, 0.37214], 18.0)","([42.1104, 48.1246, 7.27525, 0.12039, -0.07346], [21.4053, 0.13593, 0.41552], 19.0)","([42.0299, 51.6306, 19.495, 7.99422, 0.12652, -0.1076], [14.6645, 0.13897, 0.42803], 18.0)","([40.5712, 24.349, 4.50075, -43.1741, -63.8264, -0.01451, -0.01084], [45.3094, 0.1626, 0.4255], 17.0)","([36.6582, 27.4229, 49.4877, -34.442, -95.3402, 23.0829, -0.05503, 0.03389], [16.7345, 0.16613, 0.41559], 16.0)","([36.9319, 27.1027, 49.2814, -34.9325, -95.6385, 23.0628, 0.05325, -0.02511], [16.7282, 0.1659, 0.41478], 16.0)","([37.8289, 21.5044, 59.8424, -30.0277, -120.419, 28.6771, 0.18089, -0.60642], [15.8603, 0.13192, 0.43944], 16.0)","([54.3482, 0.05143, -0.0687], [13.9954, 0.15415, 0.35547], 21.0)","([40.0431, 22.574, 7.4309, -32.1725, -72.2108, 0.1357, -0.31397], [44.0554, 0.1379, 0.43501], 17.0)","([40.4083, 24.5961, 4.63499, -42.6957, -63.5959, 0.01328, 0.01657], [45.3633, 0.16273, 0.42629], 17.0)"
2,FS00008KO2,2013-10-31,"([59.1253, 0.1549, -0.66386], [13.3935, 0.14182, 0.4428], 21.0)","([57.7504, -0.08713, -0.07915], [14.6956, 0.1716, 0.37541], 21.0)","([50.8857, 0.34382, 34.9303, 0.1327, 0.05958], [24.4211, 0.18231, 0.39535], 19.0)","([53.4146, -16.456, 17.9611, -14.4733, -0.14171, -0.14923], [23.136, 0.18672, 0.43042], 18.0)","([50.5059, 0.55467, 35.0232, -0.1317, -0.05217], [24.4878, 0.18303, 0.395], 19.0)","([53.8011, -16.686, 17.7636, -14.5435, 0.14275, 0.15663], [23.0697, 0.18596, 0.43026], 18.0)","([53.7236, 1.20652, 29.6008, 0.13087, -0.63663], [23.5358, 0.1455, 0.48524], 19.0)","([56.6365, -16.6969, 11.9632, -15.0666, 0.11547, -0.73135], [22.4677, 0.14943, 0.51228], 18.0)","([48.1073, -2.21955, 50.7545, 30.3254, -27.7908, 0.15684, 0.04026], [50.1951, 0.2044, 0.48525], 17.0)","([49.0648, -5.90576, 42.5377, 29.9842, -18.5008, -5.57092, -0.16121, -0.04873], [38.4253, 0.21532, 0.5149], 16.0)","([49.4492, -6.2154, 42.5426, 29.858, -19.0283, -5.50742, 0.16233, 0.05727], [38.2411, 0.21398, 0.51361], 16.0)","([47.6621, -0.61243, 67.0324, 23.3016, -74.6261, 14.2877, 0.19336, -0.82168], [38.5431, 0.17365, 0.54209], 16.0)","([58.0151, 0.09002, 0.08271], [14.7597, 0.17134, 0.3752], 21.0)","([51.2998, -10.6169, 45.1163, 20.2713, -50.7315, 0.16566, -0.82199], [48.1875, 0.15271, 0.52816], 17.0)","([47.7021, -1.82761, 50.8303, 30.4649, -27.36, -0.15544, -0.03146], [50.31, 0.20543, 0.4863], 17.0)"
3,FS00008KO2,2013-11-30,"([56.733, 0.19529, -0.76445], [13.5991, 0.1539, 0.46906], 21.0)","([57.5681, -0.08835, -0.07226], [14.8713, 0.1712, 0.35991], 21.0)","([50.5362, 1.97152, 34.5545, 0.1294, 0.06117], [24.1165, 0.17868, 0.39558], 19.0)","([53.5011, -13.5126, 18.1243, -13.6443, -0.13331, -0.15807], [22.967, 0.18246, 0.43769], 18.0)","([50.1785, 2.21861, 34.6206, -0.128, -0.0545], [24.1724, 0.17911, 0.39457], 19.0)","([53.8694, -13.7872, 17.9457, -13.7234, 0.13478, 0.16485], [22.8966, 0.182, 0.43802], 18.0)","([49.9177, 7.18813, 28.7389, 0.17645, -0.71202], [23.1752, 0.16188, 0.49579], 19.0)","([52.9483, -8.21325, 13.1357, -13.1405, 0.15697, -0.

In [11]:
data_description

3-element Vector{Any}:
 ["Regression betas (excluding constant)"]
 ["Standard errors of currency factors only"]
 "Degrees of freedom"

Extract the info required to calculate p-values and calculate all p-values.

In [12]:
function compute_pvalues(regression_tuple)
    coef = regression_tuple.coef
    se = regression_tuple.se
    df = regression_tuple.df
    
    t = coef ./ se
    p = ccdf.(Ref(FDist(1,df)), t^2)
    
    return p
end

compute_pvalues (generic function with 1 method)

In [13]:
function build_ptable(model, source_df)
    model_col = Symbol("$(model)_betas")
    n_data = size(source_df, 1)
    empty_col = Vector{Float64}(undef, n_data)
    
    currency_factor_names = first(source_df[model]).regressors[end-2:end]
    currency_factor_columns = DataFrame(Dict(f=>copy(empty_col) for f in currency_factor_names))
    
    ptable = hcat(source_df[:, [:fundid, :date]], currency_factor_columns)
    
    for i in 1:n_data
        ptable[i, 3:4] = compute_pvalues(source_df[i, model])[end-2:end]
    end
    
    return ptable
end

build_ptable (generic function with 1 method)

This function computes the proportions of significant p-values by date.

In [14]:
truth_fraction(bool_array) = count(==(true), bool_array)/length(bool_array)

truth_fraction (generic function with 1 method)

In [15]:
function proportion_sig(ptable, sig_level)
    ptable.both_sig = (ptable[:,3] .<= sig_level) .&& (ptable[:,4] .<= sig_level)
    
    date_ptable = groupby(ptable, :date)
    sig_proportions = combine(date_ptable, :both_sig => truth_fraction => :sig_proportion)
    
    return sig_proportions
end

proportion_sig (generic function with 1 method)

This function can be called on a model name to compute the p-values, compute proportions of significant p-values by date, altogether

In [16]:
model = first(COMPLETE_MODELS)

(:lrv_net, :world_capm)

In [17]:
RESULT_COLUMNS

15-element Vector{Symbol}:
 :lrv_net_world_capm_betas
 :verdelhan_world_capm_betas
 :lrv_world_capm_betas
 :lrv_net_world_ff5_betas
 :verdelhan_world_ff5_betas
 :lrv_world_ff5_betas
 :lrv_net_world_ffcarhart_betas
 :verdelhan_world_ffcarhart_betas
 :lrv_world_ffcarhart_betas
 :lrv_net_world_ff6_betas
 :verdelhan_world_ff6_betas
 :lrv_world_ff6_betas
 :lrv_net_world_ff3_betas
 :verdelhan_world_ff3_betas
 :lrv_world_ff3_betas

In [ ]:
x = build_ptable()

In [ ]:
function build